In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 16

@author: michele ginolfi

"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import scipy.ndimage


In [ ]:
""" read from pickle """

reconstructed_df = pd.read_pickle('final_combined_dataset.pickle')

# show keys
reconstructed_df.columns

all_spectra = np.stack(reconstructed_df['combined_spectrum'].values)
# all_skyMask = np.stack(reconstructed_df['combined_skyMask'].values)
all_skyFlux = np.stack(reconstructed_df['combined_skyFlux'].values)

all_ID = reconstructed_df['ID'].values
all_exposure_times = reconstructed_df['exposure_time'].values

all_redshift = reconstructed_df['z'].values
all_stellar_masses = reconstructed_df['log_m'].values
all_sfr = np.log10(reconstructed_df['sfr'].values)

In [ ]:
""" normalize spectra """


# Applica la normalizzazione min-max
# min_val = all_spectra.min(axis=1, keepdims=True)
# max_val = all_spectra.max(axis=1, keepdims=True)
# all_spectra_normalized = (all_spectra - min_val) / (max_val - min_val)

# Applica la normalizzazione rispetto al massimo della matrice
all_spectra_normalized = all_spectra / all_spectra.max()


all_skyMask_normalized = all_skyFlux

In [ ]:
""" make X & Y datasets """

X = np.stack((all_spectra_normalized, all_skyMask_normalized), axis=-1)

Y = np.column_stack((all_redshift, all_stellar_masses, all_sfr))

In [ ]:
del reconstructed_df
del all_spectra
del all_skyFlux

In [ ]:
"""split data"""

from sklearn.model_selection import train_test_split

# First Split: Train (including validation) and Test
X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)  # 15% test

# Second Split: Train and Validation from X_temp and Y_temp
# Note: 15% of the remaining 85% is 0.1765 (approximately 17.65%)
X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=0.1765, random_state=42)  # About 15% of total

In [ ]:
""" Normalize labels """

# Calculate mean and standard deviation for each label type in the training set
Y_train_mean = Y_train.mean(axis=0)
Y_train_std = Y_train.std(axis=0)

# Normalize each label type in the training set
Y_train_normalized = (Y_train - Y_train_mean) / Y_train_std

# Normalize each label type in the validation set using training set statistics
Y_val_normalized = (Y_val - Y_train_mean) / Y_train_std

# Normalize each label type in the test set using training set statistics
Y_test_normalized = (Y_test - Y_train_mean) / Y_train_std

In [ ]:
""" Create model """

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Concatenate, BatchNormalization
import tensorflow as tf

def create_model(input_shape):

    # print(input_shape)
    # Input layer
    inputs = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(16, (1, 2),  strides=(1, 1), activation='elu')(inputs)
    # x = BatchNormalization()(x)  # Batch Normalization after convolution
    x = Conv2D(16, (100, 1), strides=(4, 1), activation='elu')(x)
    x = Conv2D(16, (2, 1), strides=(4, 1), activation='elu')(x)
    x = Flatten()(x)

    # First dense hidden layer
    # x = Dense(32, activation='relu')(inputs)

    # First dense hidden layer
    x = Dense(128, activation='elu')(x)
    # x = BatchNormalization()(x)  # Batch Normalization after dense layer
    x = Dropout(0.3)(x)  # Dropout layer

    # Second dense hidden layer
    x = Dense(64, activation='elu')(x)
    # x = BatchNormalization()(x)  # Batch Normalization after dense layer
    x = Dropout(0.3)(x)  # Dropout layer

    # Third dense hidden layer
    x = Dense(32, activation='elu')(x)
    # x = BatchNormalization()(x)  # Batch Normalization after dense layer
    x = Dropout(0.3)(x)  # Dropout layer

    # Task-specific layers
    # Redshift prediction
    redshift_output = Dense(1, activation='linear', name='redshift')(x)

    # Stellar mass prediction
    stellar_mass_output = Dense(1, activation='linear', name='stellar_mass')(x)

    # Star formation rate prediction
    sfr_output = Dense(1, activation='linear', name='sfr')(x)

    # Define model
    model = Model(inputs=inputs, outputs=[redshift_output, stellar_mass_output, sfr_output])

    return model


model = create_model(np.expand_dims(X_train[0], -1).shape) # np.expand_dims(X_train[0], -1).shape = (12217, 2, 1)

model.summary()

In [ ]:
""" Compile model """

model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'redshift': 'mse',
                    'stellar_mass': 'mse',
                    'sfr': 'mse'},
              loss_weights={'redshift': 1.0, 'stellar_mass': 1.0, 'sfr': 1.0},
              metrics={'redshift': 'mae', 'stellar_mass': 'mae', 'sfr': 'mae'})

# Early Stopping Callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_redshift_loss',  # Monitor the validation loss
    patience=10,         # Number of epochs with no improvement after which training will be stopped
    verbose=1,           # To log when training is stopped
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity.
)


# Train model with validation data
history = model.fit(X_train, {'redshift': Y_train_normalized[:, 0], 'stellar_mass': Y_train_normalized[:, 1], 'sfr': Y_train_normalized[:, 2]},
                    validation_data=(X_val, {'redshift': Y_val_normalized[:, 0], 'stellar_mass': Y_val_normalized[:, 1], 'sfr': Y_val_normalized[:, 2]}),
                    epochs=400,
                    batch_size=1024,
                    callbacks=[early_stopping])

In [ ]:
""" Check history """

def plot_history(history, task):

    plt.figure(figsize=(12, 4))

    # Plot training & validation loss values
    plt.subplot(1, 2, 1)
    plt.plot(history.history[task+'_loss'])
    plt.plot(history.history['val_'+task+'_loss'])
    plt.title('Model loss for ' + task)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history.history[task+'_mae'])
    plt.plot(history.history['val_'+task+'_mae'])
    plt.title('Model MAE for ' + task)
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    plt.show()


# plot tasks metrics
plot_history(history, 'redshift')
plot_history(history, 'stellar_mass')
plot_history(history, 'sfr')


In [ ]:
""" Check predictions on test-set """

test_metrics = model.evaluate(X_test, {'redshift': Y_test_normalized[:, 0], 'stellar_mass': Y_test_normalized[:, 1], 'sfr': Y_test_normalized[:, 2]})

# compute prediction
predictions = np.array(model.predict(X_test))

# Reshape predictions to remove the extra dimension and match Y_test
predictions_reshaped = predictions.squeeze()


def inverse_transform(normalized_values, means, stds):
    return normalized_values * stds + means

# Applying the inverse transformation to predictions
predictions_rescaled = np.array([inverse_transform(predictions[i], Y_train_mean[i], Y_train_std[i]) for i in range(predictions.shape[0])])

def plot_predictions(predicted, actual, task_name):
    plt.scatter(actual, predicted, alpha=0.1)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Predicted vs Actual Values for {task_name}')
    plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'k--', lw=4)
    plt.show()

# Plotting predictions vs actual values for each task
for i, task_name in enumerate(['Redshift', 'Stellar Mass', 'SFR']):
    plot_predictions(predictions_rescaled[i], Y_test[:, i], task_name)

In [ ]:
residuals = predictions_rescaled[0,:].squeeze() -Y_test[:, 0]
plt.hist(residuals, bins=100)

np.std(residuals)